In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('D:/projects/MGKG/data/total_triplets.csv', header=None, sep='\t', names=['head', 'relation', 'tail'])
df.shape

(3395273, 3)

### 删除df中关系数量少于20的三元组， 数量太少在知识图谱中没啥用，且影响训练集、验证集、测试集的分割

In [2]:
def omit(df):
    idx = list(df.index)
    if len(idx) < 20:
        return(idx)

In [3]:
group_df = df.groupby('relation')
omit_idx = group_df.apply(omit)
omit_index = []

for i in omit_idx.dropna():
    omit_index.extend(i)
print(len(omit_index))

df.drop(omit_index, inplace=True)
df.reset_index(drop=True, inplace=True)
print(df.shape)
group_df = df.groupby('relation')

120
(3395153, 3)


### 分割训练集、验证集、测试集

In [4]:
import math
np.random.seed(99)


def split_ttv(df):
    train_len = math.floor(df.shape[0] * 0.9)
    test_len = math.ceil(df.shape[0] * 0.05)
    # valid_len = math.ceil(df.shape[0] * 0.05)
    idx = list(df.index)
    np.random.shuffle(idx)
    train_idx = idx[: train_len]
    test_idx = idx[train_len: train_len + test_len]
    valid_idx = idx[train_len + test_len:]
    return [train_idx, test_idx, valid_idx]

In [5]:
tvt_idx = group_df.apply(split_ttv)
train_index = []
test_index = []
valid_index = []
for i in tvt_idx:
    train_index.extend(i[0])
    test_index.extend(i[1])
    valid_index.extend(i[2])
print(len(train_index), len(test_index), len(valid_index))

3055616 169777 169760


In [6]:
train_df = df.iloc[train_index, :]
test_df = df.iloc[test_index, :]
valid_df = df.iloc[valid_index, :]

# Save
train_df.to_csv('./mgkg_train.csv', sep='\t', header=0, index=0)
test_df.to_csv('./mgkg_test.csv', sep='\t', header=0, index=0)
valid_df.to_csv('./mgkg_valid.csv', sep='\t', header=0, index=0)

### 知识图谱嵌入（KGE）训练

In [ ]:
# Linux命令行
# TransR Model
!DGLBACKEND=pytorch dglke_train --dataset MGKG --data_path ./Train --data_files mgkg_train.csv mgkg_valid.csv mgkg_test.csv --format 'raw_udd_hrt' --model_name TransR --batch_size 2048 --neg_sample_size 256 --hidden_dim 400 --gamma 12.0 --lr 0.1 --max_step 100000 --log_interval 1000 --batch_size_eval 16 -adv --regularization_coef 1.00E-07 --test --num_thread 1 --gpu 0 --neg_sample_size_eval 10000

# TransE_l2 Model
!DGLBACKEND=pytorch dglke_train --dataset MGKG --data_path ./Train --data_files mgkg_train.csv mgkg_valid.csv mgkg_test.csv --format 'raw_udd_hrt' --model_name TransE_l2 --batch_size 2048 --neg_sample_size 256 --hidden_dim 400 --gamma 12.0 --lr 0.1 --max_step 100000 --log_interval 1000 --batch_size_eval 16 -adv --regularization_coef 1.00E-07 --test --num_thread 1 --gpu 0 --neg_sample_size_eval 10000